# Feature Engineering

In [ ]:
import numpy as np
import pandas as pd

from collections import Counter

In [39]:
player_data = pd.read_csv('ncaa_data/MPlayers.csv')
events_2019 = pd.read_csv('ncaa_data/MEvents2019.csv')
regular_season = pd.read_csv('ncaa_data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
season = pd.read_csv('ncaa_data/MDataFiles_Stage1/MSeasons.csv')
seeds = pd.read_csv('ncaa_data/MDataFiles_Stage1/MNCAATourneySeeds.csv')
tournament_results = pd.read_csv('ncaa_data/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
teams = pd.read_csv('ncaa_data/MDataFiles_Stage1/MTeams.csv')

In [24]:
tournament_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT'],
      dtype='object')

In [40]:
teams.drop(['FirstD1Season', 'LastD1Season'], inplace = True, axis = 1)

In [41]:
winning_team = teams.rename(columns = {'TeamID': 'WTeamID', 'TeamName': 'WTeamName'})
losing_team = teams.rename(columns = {'TeamID': 'LTeamID', 'TeamName': 'LTeamName'})
data = pd.merge(left = regular_season, right = winning_team, how = 'left', on = ['WTeamID'])
regular_season = pd.merge(left = data, right = losing_team, on = ['LTeamID'])

### Each Teams Regular Season Wins

In [225]:
seasons_list = list(range(1985,2020))
teams_df = []
wins_df = []
seasons_df = []

for season in seasons_list:
    df_season = regular_season[regular_season.Season == season]
    Wteam_list = df_season.WTeamID.values.tolist()
    counts = dict()
    for i in Wteam_list:
        counts[i] = counts.get(i,0)+1
    team_list = list(counts.keys())
    wins_list = list(counts.values())
    teams_df.append(team_list)
    wins_df.append(wins_list)
    seasons_df.append([season]*len(team_list))
    
teams_df = [item for sublist in teams_df for item in sublist]
wins_df = [item for sublist in wins_df for item in sublist]
seasons_df = [item for sublist in seasons_df for item in sublist]
data = {'Season': seasons_df, 'TeamID': teams_df, 'RegularSeasonWins': wins_df}
regular_season_wins = pd.DataFrame(data)

### Seed Differencing

In [228]:
# Function that removes letter from seeds
def seed_num(seed):
    seed_number = int(seed[1:3])
    return seed_number
seeds['Seed'] = seeds.Seed.apply(seed_num)
seeds.head()

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374


In [230]:
winners = seeds.rename(columns = {'TeamID': 'WTeamID', 'Seed':'WSeed'})
losers = seeds.rename(columns = {'TeamID': 'LTeamID', 'Seed': 'LSeed'})
tournament_winners = pd.merge(left=tournament_results, right=winners, how = 'left', on =['Season', 'WTeamID'])
tournament = pd.merge(left= tournament_winners, right=losers, on =['Season', 'LTeamID'])

tournament['Seed_Difference'] = tournament.LSeed - tournament.WSeed
tournament.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSeed,LSeed,Seed_Difference
0,1985,136,1116,63,1234,54,N,0,9,8,-1
1,1985,136,1120,59,1345,58,N,0,11,6,-5
2,1985,136,1207,68,1250,43,N,0,1,16,15
3,1985,136,1229,58,1425,55,N,0,9,8,-1
4,1985,136,1242,49,1325,38,N,0,3,14,11


In [235]:
tournament['Results'] = np.where(tournament['WSeed'] < tournament['LSeed'], 1, 0)  
tournament.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WSeed,LSeed,Seed_Difference,Results
0,1985,136,1116,63,1234,54,N,0,9,8,-1,0
1,1985,136,1120,59,1345,58,N,0,11,6,-5,0
2,1985,136,1207,68,1250,43,N,0,1,16,15,1
3,1985,136,1229,58,1425,55,N,0,9,8,-1,0
4,1985,136,1242,49,1325,38,N,0,3,14,11,1
